## 1.

*Recogiendo los datos de coronavirus de la página https://ourworldindata.org/coronavirus-source-data* 

A partir de la letra de tu DNI, elige al país que empieza por dicha letra que más fallecidos tenga. Si el país elegido no está entre el top 20 de fallecidos, elige como país España.

En este caso, utilizaremos las columnas:

- Nuevos casos
- Nuevos fallecidos
- Población
- Fallecidos totales
- Casos totales

La fecha debe estar en el índice.

1. Representa todas las columnas en un gráfica con diferentes magnitudes en el eje y para su correcta visualización.

2. Elimina la columna "Población".

3. Entrena los algoritmos de regresión que hemos visto hasta ahora para predecir dos columnas por separado: Nuevos casos y Nuevos fallecidos. Este modelo ha de entrenarse siguiendo el esquema recomendado de validación cruzada y almacenando los errores de entrenamiento y validación en cada iteración del entrenamiento. ¿Qué error se comete? ¿crees que los algoritmos de ML vistos hasta ahora son buenos para realizar un forecast de una serie temporal viendo los resultados?

4. Representa la evolución de los errores de validación y entrenamiento en una gráfica. ¿Ha habido sobreaprendizaje?. Utiliza n_iteraciones = 23 y n_repeats = 7 para este ejercicio y solo para la columna "nuevos casos".

In [1]:
# Data
import pandas as pd
import numpy as np

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Machine Learning

from sklearn import model_selection
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, LassoCV
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.pipeline import Pipeline

In [3]:
# Thailand
covid = pd.read_csv("owid-covid-data.csv", index_col= 'date')

In [4]:
covid_seg = covid[covid['location'] == 'Thailand'][['population','new_cases','new_deaths','total_deaths', 'total_cases']]

In [5]:
covid_seg.head()

,population,new_cases,new_deaths,total_deaths,total_cases
date,,,,,
2020-01-04,69799978.0,NaN,NaN,NaN,NaN
2020-01-05,69799978.0,NaN,NaN,NaN,NaN
2020-01-06,69799978.0,NaN,NaN,NaN,NaN
2020-01-07,69799978.0,NaN,NaN,NaN,NaN
2020-01-08,69799978.0,NaN,NaN,NaN,NaN


In [6]:
covid_seg.isnull().sum()/len(covid_seg)*100

population       0.000000
new_cases        3.598485
new_deaths      10.795455
total_deaths    10.795455
total_cases      3.409091
dtype: float64

In [7]:
covid_seg.dropna(inplace = True)

In [8]:
covid_seg

,population,new_cases,new_deaths,total_deaths,total_cases
date,,,,,
2020-03-01,69799978.0,0.0,1.0,1.0,42.0
2020-03-02,69799978.0,1.0,0.0,1.0,43.0
2020-03-03,69799978.0,0.0,0.0,1.0,43.0
2020-03-04,69799978.0,0.0,0.0,1.0,43.0
2020-03-05,69799978.0,4.0,0.0,1.0,47.0
...,...,...,...,...,...
2021-06-10,69799978.0,2310.0,43.0,1375.0,187538.0
2021-06-11,69799978.0,2290.0,27.0,1402.0,189828.0
2021-06-12,69799978.0,3277.0,29.0,1431.0,193105.0


### 1. Representa todas las columnas en un gráfica con diferentes magnitudes en el eje y para su correcta visualización.



In [9]:
dicti = {"axis_1" : "new_cases",
        "axis_2" : "new_deaths",
        "axis_3" : "population",
        "axis_4" : "total_deaths",
        "axis_5" : "total_cases"}

In [10]:
colors = px.colors.qualitative.Plotly

In [12]:
fig = go.Figure()

for k,v in dicti.items():
    fig.add_trace(go.Scatter(
        x=covid_seg.index,
        y=covid_seg[v],
        name= v,
        yaxis = "y" + str(k[-1])))
    

# Create axis objects
fig.update_layout(
    xaxis=dict(
        domain=[0.12, 0.5]
    ),
    yaxis=dict(
        title="yaxis1",
        titlefont=dict(
            color=colors[0]
        ),
        tickfont=dict(
            color=colors[0]
        )
    ),
    yaxis2=dict(
        title="yaxis2",
        titlefont=dict(
            color=colors[1]
        ),
        tickfont=dict(
            color=colors[1]
        ),
        anchor="free",
        overlaying="y",
        side="left",
        position=0.05
    ),
    yaxis3=dict(
        title="yaxis3",
        titlefont=dict(
            color=colors[2]
        ),
        tickfont=dict(
            color=colors[2]
        ),anchor="free",
        overlaying="y",
        side="right",
        position=0.56
        
    ),   
    yaxis4=dict(
        title="yaxis4",
        titlefont=dict(
            color=colors[3]
        ),
        tickfont=dict(
            color=colors[3]
        ),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    yaxis5=dict(
        title="yaxis5",
        titlefont=dict(
            color=colors[4]
        ),
        tickfont=dict(
            color=colors[4]
        ),
        anchor="free",
        overlaying="y",
        side="left",
        position=0
    )
)

# Update layout properties
fig.update_layout(
    title_text= "Covid in Thailand",
    width=1500,
    legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.12
)
)

fig.show()

### 2. Elimina la columna "Población".



In [13]:
covid_seg.drop(columns = "population", inplace = True)

In [14]:
covid_seg.head()

,new_cases,new_deaths,total_deaths,total_cases
date,,,,
2020-03-01,0.0,1.0,1.0,42.0
2020-03-02,1.0,0.0,1.0,43.0
2020-03-03,0.0,0.0,1.0,43.0
2020-03-04,0.0,0.0,1.0,43.0
2020-03-05,4.0,0.0,1.0,47.0


### 3. Entrena los algoritmos de regresión que hemos visto hasta ahora para predecir dos columnas por separado: Nuevos casos y Nuevos fallecidos. Este modelo ha de entrenarse siguiendo el esquema recomendado de validación cruzada y almacenando los errores de entrenamiento y validación en cada iteración del entrenamiento. ¿Qué error se comete? ¿crees que los algoritmos de ML vistos hasta ahora son buenos para realizar un forecast de una serie temporal viendo los resultados?



#### Nuevos casos

In [15]:
X = covid_seg.drop(columns = ['new_cases']).values
y = covid_seg['new_cases'].values

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

k_fold = RepeatedKFold(n_splits=23, n_repeats=7)

In [17]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((376, 3), (95, 3), (376,), (95,))

In [18]:
dictionary_model_reg = {"poly_model": Pipeline([
            ("polynomial_features", PolynomialFeatures(3)),
            ("linear_regression", LinearRegression())]),
                    "linear_model": LinearRegression(),
                    "support_model": SVR(),
                    "random_forest": RandomForestRegressor(),
                    "XGBoost": XGBRegressor()
                   }


In [ ]:
error_dicti = {"model": [], "iteracion":[],"train_error":[], "val_error":[]}
for name, model in dictionary_model_reg.items(): 
    for i, (train, val) in enumerate(k_fold.split(X_train)):
        error_dicti['model'].append(name)
        print("Iteración:", i+1)
        error_dicti['iteracion'].append(i+1)
        print("batch_size:", len(train))
        print("val_size:", len(val))
        print("batch:", len(X_train[train]))
        print("val:", val)

        model.fit(X_train[train], y_train[train])


        y_pred_train = model.predict(X_train[train])
        train_error = mean_absolute_error(y_train[train], y_pred_train)
        
        print("MAE of train:", train_error)
        error_dicti['train_error'].append(train_error)

        y_pred_val = model.predict(X_train[val])
        error_val = mean_absolute_error(y_train[val], y_pred_val)
        print("MAE of validation:", error_val)
        error_dicti['val_error'].append(error_val)
        print('#####################')

Iteración: 1
batch_size: 359
val_size: 17
batch: 359
val: [ 22  40  42 107 113 115 136 149 156 161 169 197 211 220 224 295 307]
MAE of train: 121.79677084608076
MAE of validation: 77.00278575178795
#####################
Iteración: 2
batch_size: 359
val_size: 17
batch: 359
val: [  4   6  17  67  76 146 175 189 225 234 235 240 255 276 283 343 363]
MAE of train: 121.80253008248313
MAE of validation: 107.63219952045267
#####################
Iteración: 3
batch_size: 359
val_size: 17
batch: 359
val: [ 10  13  18  25  45  68  70 137 182 210 233 281 287 294 330 359 375]
MAE of train: 117.978633202103
MAE of validation: 151.34956269462697
#####################
Iteración: 4
batch_size: 359
val_size: 17
batch: 359
val: [ 26  28  35  39  62  93 121 124 129 154 179 247 250 251 270 289 341]
MAE of train: 122.72023981084811
MAE of validation: 62.82238190902335
#####################
Iteración: 5
batch_size: 359
val_size: 17
batch: 359
val: [  7  12 103 120 144 159 176 196 203 205 259 312 316 320 339 3

MAE of train: 113.16722188660086
MAE of validation: 207.41567587104822
#####################
Iteración: 60
batch_size: 360
val_size: 16
batch: 360
val: [ 14  16  54  75 134 164 226 227 235 238 242 271 294 306 318 349]
MAE of train: 121.92716147609893
MAE of validation: 62.69309208545836
#####################
Iteración: 61
batch_size: 360
val_size: 16
batch: 360
val: [  7  51  61  68  73  80 117 142 144 166 169 254 307 320 336 371]
MAE of train: 118.16365298355227
MAE of validation: 154.26849508747227
#####################
Iteración: 62
batch_size: 360
val_size: 16
batch: 360
val: [ 24  31  71  95  96 138 183 213 244 277 288 303 313 314 328 332]
MAE of train: 114.67490057259893
MAE of validation: 515.4828964299613
#####################
Iteración: 63
batch_size: 360
val_size: 16
batch: 360
val: [  0 172 185 200 218 222 239 256 283 309 312 315 319 329 343 354]
MAE of train: 121.64022377460152
MAE of validation: 84.45795765604264
#####################
Iteración: 64
batch_size: 360
val_size

MAE of train: 119.1358437997276
MAE of validation: 121.43317925950132
#####################
Iteración: 118
batch_size: 359
val_size: 17
batch: 359
val: [  9  33  82  96 105 121 125 141 150 156 170 214 266 277 319 338 348]
MAE of train: 118.73015176928035
MAE of validation: 113.20824753045915
#####################
Iteración: 119
batch_size: 359
val_size: 17
batch: 359
val: [ 23  79 129 137 140 143 152 154 163 168 187 203 215 230 256 296 370]
MAE of train: 123.10920583988836
MAE of validation: 43.08932314048312
#####################
Iteración: 120
batch_size: 359
val_size: 17
batch: 359
val: [ 28  31  72  91  93 138 179 196 201 202 213 222 240 330 332 336 350]
MAE of train: 119.13386728882575
MAE of validation: 159.73421903621752
#####################
Iteración: 121
batch_size: 359
val_size: 17
batch: 359
val: [ 34  43  60  67  71  84 100 106 153 180 186 221 257 271 276 280 312]
MAE of train: 119.32704457122885
MAE of validation: 120.26704878842529
#####################
Iteración: 122
ba

MAE of train: 162.71331792732195
MAE of validation: 241.905996233216
#####################
Iteración: 29
batch_size: 359
val_size: 17
batch: 359
val: [ 30  45  83  86  90 107 190 213 214 234 243 252 333 334 358 360 371]
MAE of train: 170.07435601828234
MAE of validation: 63.83044438774965
#####################
Iteración: 30
batch_size: 359
val_size: 17
batch: 359
val: [ 10  61  64  67  95 100 147 173 197 230 233 248 292 295 314 321 366]
MAE of train: 162.68213083363023
MAE of validation: 212.6365711274048
#####################
Iteración: 31
batch_size: 359
val_size: 17
batch: 359
val: [  3   8  22  38  48 114 122 189 201 207 223 251 256 287 346 359 364]
MAE of train: 166.08089933556107
MAE of validation: 194.5728699716336
#####################
Iteración: 32
batch_size: 360
val_size: 16
batch: 360
val: [ 34  37  62 134 161 162 166 241 255 257 260 275 336 367 369 370]
MAE of train: 164.6592081960256
MAE of validation: 226.62858531051074
#####################
Iteración: 33
batch_size: 360

Iteración: 70
batch_size: 359
val_size: 17
batch: 359
val: [ 20  21  22  50  90 120 124 204 232 233 249 270 298 302 318 325 351]
MAE of train: 166.0896081252026
MAE of validation: 215.41619754244826
#####################
Iteración: 71
batch_size: 359
val_size: 17
batch: 359
val: [ 13  94 126 130 163 199 227 238 242 257 259 265 290 313 342 350 359]
MAE of train: 164.6895144594419
MAE of validation: 177.59116916920104
#####################
Iteración: 72
batch_size: 359
val_size: 17
batch: 359
val: [  1  12  56 123 135 150 180 189 200 210 213 239 280 291 314 326 369]
MAE of train: 171.51334832729603
MAE of validation: 69.86495368839462
#####################
Iteración: 73
batch_size: 359
val_size: 17
batch: 359
val: [  6  28  30  74  87  93 131 161 191 194 203 208 225 277 297 364 368]
MAE of train: 169.75305707779344
MAE of validation: 135.979786930409
#####################
Iteración: 74
batch_size: 359
val_size: 17
batch: 359
val: [ 16  38  39  70 106 114 156 171 186 192 202 229 235 256 3

...

MAE of train: 40.626654863397206
MAE of validation: 129.79201400560223
#####################
Iteración: 6
batch_size: 359
val_size: 17
batch: 359
val: [  3  19  94  95 148 200 252 278 282 283 298 312 319 341 347 352 364]
MAE of train: 38.45986458195163
MAE of validation: 113.49466666666667
#####################
Iteración: 7
batch_size: 359
val_size: 17
batch: 359
val: [  2  27  35  36  60  77  87  92 152 170 176 180 195 225 239 306 365]
MAE of train: 39.34674953574744
MAE of validation: 87.25491596638656
#####################
Iteración: 8
batch_size: 359
val_size: 17
batch: 359
val: [ 41  54 101 107 112 134 145 161 246 251 279 296 303 317 340 344 367]
MAE of train: 38.5189219613565
MAE of validation: 102.36820588235295
#####################
Iteración: 9
batch_size: 360
val_size: 16
batch: 360
val: [  8  68  76  83 109 141 143 177 212 223 267 273 275 302 334 372]
MAE of train: 39.66840033068783
MAE of validation: 30.724922619047618
#####################
Iteración: 10
batch_size: 360
val

MAE of train: 37.43113167989417
MAE of validation: 194.93575
#####################
Iteración: 44
batch_size: 360
val_size: 16
batch: 360
val: [ 39  40  56  81  94 128 142 143 155 180 190 235 264 278 283 359]
MAE of train: 40.18508022486772
MAE of validation: 38.424458333333334
#####################
Iteración: 45
batch_size: 360
val_size: 16
batch: 360
val: [  4   7  19  62 132 179 189 192 212 222 245 253 311 316 321 342]
MAE of train: 38.94599535934744
MAE of validation: 102.26993154761904
#####################
Iteración: 46
batch_size: 360
val_size: 16
batch: 360
val: [ 25  34  75  84 161 162 166 177 198 234 249 252 261 265 280 323]
MAE of train: 39.47373917548501
MAE of validation: 117.48901041666667
#####################
Iteración: 47
batch_size: 359
val_size: 17
batch: 359
val: [ 19  23  62  72 182 218 226 235 252 253 258 260 270 296 319 324 341]
MAE of train: 40.48013242251403
MAE of validation: 63.995927170868356
#####################
Iteración: 48
batch_size: 359
val_size: 17
ba

MAE of train: 40.10649050925926
MAE of validation: 81.52971726190476
#####################
Iteración: 82
batch_size: 360
val_size: 16
batch: 360
val: [ 15  21  30  48  76 106 117 148 215 219 220 271 309 335 370 374]
MAE of train: 39.868226267636686
MAE of validation: 38.73887946428572
#####################
Iteración: 83
batch_size: 360
val_size: 16
batch: 360
val: [ 10  14  78  98 101 111 125 142 144 166 178 261 285 301 312 334]
MAE of train: 34.63307946829405
MAE of validation: 270.4601041666667
#####################
Iteración: 84
batch_size: 360
val_size: 16
batch: 360
val: [  0   6  38  41  89 141 156 204 236 291 300 320 325 332 333 362]
MAE of train: 40.51545181878306
MAE of validation: 85.41337946428571
#####################
Iteración: 85
batch_size: 360
val_size: 16
batch: 360
val: [ 33  69 118 124 163 175 200 210 246 294 299 313 340 341 347 350]
MAE of train: 39.228892923280426
MAE of validation: 30.69311904761905
#####################
Iteración: 86
batch_size: 360
val_size: 16


MAE of train: 40.41580864353845
MAE of validation: 41.23083333333332
#####################
Iteración: 120
batch_size: 359
val_size: 17
batch: 359
val: [ 46  85  96  98 137 157 175 179 194 212 214 218 274 288 299 351 354]
MAE of train: 39.57629410399257
MAE of validation: 74.90354621848739
#####################
Iteración: 121
batch_size: 359
val_size: 17
batch: 359
val: [  5  10  81 103 182 186 195 201 216 249 251 269 290 300 301 340 352]
MAE of train: 38.00174962819601
MAE of validation: 192.9675798319328
#####################
Iteración: 122
batch_size: 359
val_size: 17
batch: 359
val: [ 41  75 105 107 129 142 153 177 185 229 234 244 257 284 291 305 339]
MAE of train: 41.94246474896195
MAE of validation: 36.677232492997206
#####################
Iteración: 123
batch_size: 359
val_size: 17
batch: 359
val: [ 11  19  37  64  65  77  84 114 161 162 173 184 193 238 318 335 350]
MAE of train: 40.65222734005395
MAE of validation: 144.9746176470588
#####################
Iteración: 124
batch_siz

MAE of validation: 99.61172916666666
#####################
Iteración: 158
batch_size: 360
val_size: 16
batch: 360
val: [ 82  94  97 120 121 156 248 259 290 308 318 321 333 335 341 373]
MAE of train: 38.24454603174603
MAE of validation: 69.31453720238096
#####################
Iteración: 159
batch_size: 360
val_size: 16
batch: 360
val: [  6  41  42  74  85 102 118 136 166 167 172 206 265 270 282 369]
MAE of train: 38.48255836640212
MAE of validation: 82.2729538690476
#####################
Iteración: 160
batch_size: 360
val_size: 16
batch: 360
val: [  5  30  52  58  95 109 126 129 149 154 186 223 264 304 328 356]
MAE of train: 28.772983526735608
MAE of validation: 434.19315289051224
#####################
Iteración: 161
batch_size: 360
val_size: 16
batch: 360
val: [ 11  14  17  39  63  91 119 225 226 239 272 288 294 311 339 350]
MAE of train: 40.973929728835984
MAE of validation: 40.575458333333344
#####################
Iteración: 1
batch_size: 359
val_size: 17
batch: 359
val: [  9  50  58

In [ ]:
error = pd.DataFrame(error_dicti)

In [ ]:
error.groupby(["iteracion", "model"]).max()

### 4. Representa la evolución de los errores de validación y entrenamiento en una gráfica. ¿Ha habido sobreaprendizaje?. Utiliza n_iteraciones = 23 y n_repeats = 7 para este ejercicio y solo para la columna "nuevos casos".

### for more info: https://plotly.com/python/ml-regression/